In [1]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D, BatchNormalization, Lambda
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [3]:
def get_num_classes():
    persons = os.listdir('data')
    num_classes = len(persons)
    return num_classes

In [4]:
def create_training_data(target_size):
    images = []
    labels = []
    persons = os.listdir('data')
    num_classes = len(persons)

    for person_index, person in enumerate(persons):
        folder_path = os.path.join('data', person)
        person_images = load_images_from_folder(folder_path)
        resized_images = [cv2.resize(img, target_size) for img in person_images]
        images.extend(resized_images)
        labels.extend([person_index] * len(resized_images))

    images = np.array(images)
    labels = np.array(labels)

    return images, labels, num_classes

In [5]:
model = load_model('model/facenet_keras.h5')
model.load_weights('model/facenet_keras_weights.h5')
model.layers.pop()
last_layer_output = model.layers[-1].output
num_classes = get_num_classes()
output_layer = Dense(num_classes, activation='softmax')(last_layer_output)
model = Model(inputs=model.input, outputs=output_layer)

In [6]:
target_size = (160, 160)
images, labels, num_classes = create_training_data(target_size)

x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
y_train = np_utils.to_categorical(y_train, num_classes)
y_val = np_utils.to_categorical(y_val, num_classes)
x_train = x_train / 255.0
x_val = x_val / 255.0

learning_rate = 0.001
batch_size = 32
epochs = 30

optimizer = Adam(learning_rate=learning_rate)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'], run_eagerly=True)

datagen_train = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                                   horizontal_flip=True)

datagen_train.fit(x_train)

In [7]:
checkpoint = ModelCheckpoint('model/facenet_retrained.h5', monitor='val_accuracy', verbose=1,
                             save_best_only=True, mode='max')

history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_val, y_val), 
    callbacks=[checkpoint]
)

Epoch 1/30
1/1 [==============================] - ETA: 0s - loss: 0.9656 - accuracy: 0.5294
Epoch 1: val_accuracy improved from -inf to 0.40000, saving model to model\facenet_retrained.h5
1/1 [==============================] - 4s 4s/step - loss: 0.9656 - accuracy: 0.5294 - val_loss: 1.0999 - val_accuracy: 0.4000
Epoch 2/30
1/1 [==============================] - ETA: 0s - loss: 0.1878 - accuracy: 0.8824
Epoch 2: val_accuracy did not improve from 0.40000
1/1 [==============================] - 3s 3s/step - loss: 0.1878 - accuracy: 0.8824 - val_loss: 0.9422 - val_accuracy: 0.4000
Epoch 3/30
1/1 [==============================] - ETA: 0s - loss: 0.0324 - accuracy: 1.0000
Epoch 3: val_accuracy did not improve from 0.40000
1/1 [==============================] - 3s 3s/step - loss: 0.0324 - accuracy: 1.0000 - val_loss: 0.9658 - val_accuracy: 0.4000
Epoch 4/30
1/1 [==============================] - ETA: 0s - loss: 0.0057 - accuracy: 1.0000
Epoch 4: val_accuracy did not improve from 0.40000
1/1 [

1/1 [==============================] - ETA: 0s - loss: 6.2658e-05 - accuracy: 1.0000
Epoch 30: val_accuracy did not improve from 1.00000
1/1 [==============================] - 2s 2s/step - loss: 6.2658e-05 - accuracy: 1.0000 - val_loss: 0.3828 - val_accuracy: 0.8000


In [8]:
test_images, test_labels, _ = create_training_data(target_size)
test_images = test_images / 255.0
test_labels = np_utils.to_categorical(test_labels, num_classes)

model.load_weights('model/facenet_retrained.h5')
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

Test Loss: 0.25548774003982544
Test Accuracy: 0.9545454382896423


In [9]:
image_path = 'test/test_1.jpg'
image = cv2.imread(image_path)
image = cv2.resize(image, (160, 160))
image = np.expand_dims(image, axis=0)
image = image / 255.0
predictions = model.predict(image)
persons = os.listdir('data')
predicted_person_index = np.argmax(predictions[0])
predicted_person = persons[predicted_person_index]
print(predicted_person)

1/1 [==============================] - 3s 3s/step
Ilya


In [10]:
image_path = 'test/test_2.jpg'
image = cv2.imread(image_path)
image = cv2.resize(image, (160, 160))
image = np.expand_dims(image, axis=0)
image = image / 255.0
predictions = model.predict(image)
persons = os.listdir('data')
predicted_person_index = np.argmax(predictions[0])
predicted_person = persons[predicted_person_index]
print(predicted_person)

1/1 [==============================] - 0s 90ms/step
Ksyusha
